In [1]:
using Pkg, JuMP, Ipopt, DataFrames, CSV, Random, Distributions, Statistics, HypothesisTests

In [10]:

num_iterations = 1


cweights = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/train_plyr_weights_0814.csv")
pweights = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/train_course_weights_0814.csv")


,Column1,course,c_idx,weight
,Int64,String,Int64,Float64
1,0,Bay Hill Club & Lodge,2,0.0296479
2,1,Bethpage State Park BK Course,5,0.0366739
3,2,CC of Jackson,6,0.0314237
4,3,Club de Golf Chapultepec,8,0.0221587
5,4,Colonial CC,9,0.0297251
6,5,Detroit Golf Club,14,0.0348981
7,6,East Lake GC,16,0.00926498
8,7,Hamilton Golf & Country Club,21,0.0350525
9,8,Harbour Town GL,22,0.0308833


In [11]:
# CHANGED WEIGHTS TO BE NEGATIVE BECAUSE NEW FORMULA

function get_weights(df, pweight_df = pweights,cweight_df = cweights)
    n = nrow(df)
    p_d=Dict([(i,count(x->x==i,df[!,:X_p_y_idx])/n) for i in unique(df[!,:X_p_y_idx])])
    p_weight = DataFrame(X_p_y_idx = collect(keys(p_d)), weight = collect(values(p_d)))

    c_d = Dict([(i,count(x->x==i,df[!,:X_c_idx])/n) for i in unique(df[!,:X_c_idx])])
    c_weight = DataFrame(X_c_idx = collect(keys(c_d)), weight = collect(values(c_d)))
    
    
    
    return p_weight, c_weight
end

function get_weights_nodsr(df, pweight_df = pweights,cweight_df = cweights)
    #println("test1")
    n = nrow(df)
    p_d=Dict([(i,count(x->x==i,df[!,:X_p_y_idx])/n) for i in unique(df[!,:X_p_y_idx])])
    p_weight = DataFrame(X_p_y_idx = collect(keys(p_d)), weight = collect(values(p_d)))
    #println("test2")
    c_d = Dict([(i,count(x->x==i,df[!,:X_c_idx])/n) for i in unique(df[!,:X_c_idx])])
    c_weight = DataFrame(X_c_idx = collect(keys(c_d)), weight = collect(values(c_d)))
    #println("test3")
    
    return p_weight, c_weight
end


function test_rmse( p_sg , c_sg , c_dsr , data )
    pred_data = join(data, p_sg, on = :p_y_idx, makeunique=true)
    pred_data = join(pred_data, c_sg, on = :c_idx, makeunique=true)
    pred_data = join(pred_data, c_dsr, on = :c_idx, makeunique=true)
    
    pred_data[:SG_drive_pred] = (pred_data[:sg_drive].*pred_data[:dsr_drive] .+ pred_data[:sg_drive_1]) .- pred_data[:SG_drive]
    pred_data[:SG_appr_pred] = (pred_data[:sg_appr].*pred_data[:dsr_appr] .+ pred_data[:sg_appr_1]).- pred_data[:SG_appr]
    pred_data[:SG_short_pred] = (pred_data[:sg_short].*pred_data[:dsr_short] .+ pred_data[:sg_short_1]).- pred_data[:SG_short]
    pred_data[:SG_putt_pred] = (pred_data[:sg_putt].*pred_data[:dsr_putt] .+ pred_data[:sg_putt_1]).- pred_data[:SG_putt]

    
    pred_data[:pred_sq_error] = pred_data[:SG_drive_pred].*pred_data[:SG_drive_pred] .+ pred_data[:SG_appr_pred].*pred_data[:SG_appr_pred] .+ pred_data[:SG_short_pred].*pred_data[:SG_short_pred] .+ pred_data[:SG_putt_pred].*pred_data[:SG_putt_pred]
    
    rmse = sqrt(mean(pred_data[:pred_sq_error]))/2
    drive_rmse = sqrt(mean(pred_data[:SG_drive_pred].*pred_data[:SG_drive_pred]))/2
    appr_rmse = sqrt(mean(pred_data[:SG_appr_pred].*pred_data[:SG_appr_pred]))/2
    short_rmse = sqrt(mean(pred_data[:SG_short_pred].*pred_data[:SG_short_pred]))/2
    putt_rmse = sqrt(mean(pred_data[:SG_putt_pred].*pred_data[:SG_putt_pred]))/2
    
    return rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse,  pred_data[:pred_sq_error]
end


function test_rmse_nodsr( p_sg , c_sg  , data )
    pred_data = join(data, p_sg, on = :p_y_idx, makeunique=true)
    pred_data = join(pred_data, c_sg, on = :c_idx, makeunique=true)
    
    pred_data[:SG_drive_pred] = (pred_data[:sg_drive] .+ pred_data[:sg_drive_1]) .- pred_data[:SG_drive]
    pred_data[:SG_appr_pred] = (pred_data[:sg_appr] .+ pred_data[:sg_appr_1]).- pred_data[:SG_appr]
    pred_data[:SG_short_pred] = (pred_data[:sg_short] .+ pred_data[:sg_short_1]).- pred_data[:SG_short]
    pred_data[:SG_putt_pred] = (pred_data[:sg_putt] .+ pred_data[:sg_putt_1]).- pred_data[:SG_putt]
    
    pred_data[:pred_sq_error] = pred_data[:SG_drive_pred].*pred_data[:SG_drive_pred] .+ pred_data[:SG_appr_pred].*pred_data[:SG_appr_pred] .+ pred_data[:SG_short_pred].*pred_data[:SG_short_pred] .+ pred_data[:SG_putt_pred].*pred_data[:SG_putt_pred]
    rmse = sqrt(mean(pred_data[:pred_sq_error]))/2
    drive_rmse = sqrt(mean(pred_data[:SG_drive_pred].*pred_data[:SG_drive_pred]))/2
    appr_rmse = sqrt(mean(pred_data[:SG_appr_pred].*pred_data[:SG_appr_pred]))/2
    short_rmse = sqrt(mean(pred_data[:SG_short_pred].*pred_data[:SG_short_pred]))/2
    putt_rmse = sqrt(mean(pred_data[:SG_putt_pred].*pred_data[:SG_putt_pred]))/2
    
    return rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse,  pred_data[:pred_sq_error]
end

function get_avg_ests(df, p_avgs = pweights,c_avgs = cweights)
    n = nrow(df)
    p_ests = DataFrame(p_y_idx = unique(df[!,:p_y_idx]))
    c_ests = DataFrame(c_idx = unique(df[!,:c_idx]))

    p_drive_cnstr = Dict([(i,p_avgs[!,:sg_drive_mean][i+1]) for i in unique(df[!,:p_y_idx])])
    p_appr_cnstr = Dict([(i,p_avgs[!,:sg_appr_mean][i+1]) for i in unique(df[!,:p_y_idx])])
    p_short_cnstr = Dict([(i,p_avgs[!,:sg_short_mean][i+1]) for i in unique(df[!,:p_y_idx])])
    p_putt_cnstr = Dict([(i,p_avgs[!,:sg_putt_mean][i+1]) for i in unique(df[!,:p_y_idx])])

    p_ests[!,:sg_drive_mean] = collect(values(p_drive_cnstr))
    p_ests[!,:sg_appr_mean] = collect(values(p_appr_cnstr))
    p_ests[!,:sg_short_mean] = collect(values(p_short_cnstr))
    p_ests[!,:sg_putt_mean] = collect(values(p_putt_cnstr))
    
    c_drive_cnstr = Dict([(i,-c_avgs[!,:sg_drive_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_appr_cnstr = Dict([(i,-c_avgs[!,:sg_appr_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_short_cnstr = Dict([(i,-c_avgs[!,:sg_short_mean][i+1]) for i in unique(df[!,:c_idx])])
    c_putt_cnstr = Dict([(i,-c_avgs[!,:sg_putt_mean][i+1]) for i in unique(df[!,:c_idx])])
    
    c_ests[!,:sg_drive_mean] = collect(values(c_drive_cnstr))
    c_ests[!,:sg_appr_mean] = collect(values(c_appr_cnstr))
    c_ests[!,:sg_short_mean] = collect(values(c_short_cnstr))
    c_ests[!,:sg_putt_mean] = collect(values(c_putt_cnstr))
    
    
    
    return p_ests, c_ests
end

function test_rmse_avg( p_sg , c_sg  , data )
    pred_data = join(data, p_sg, on = :p_y_idx, makeunique=true)
    pred_data = join(pred_data, c_sg, on = :c_idx, makeunique=true)
    
    pred_data[:SG_drive_pred] = (pred_data[:sg_drive_mean] .+ pred_data[:sg_drive_mean_1]) .- pred_data[:SG_drive]
    pred_data[:SG_appr_pred] = (pred_data[:sg_appr_mean] .+ pred_data[:sg_appr_mean_1]) .- pred_data[:SG_appr]
    pred_data[:SG_short_pred] = (pred_data[:sg_short_mean] .+ pred_data[:sg_short_mean_1]) .- pred_data[:SG_short]
    pred_data[:SG_putt_pred] = (pred_data[:sg_putt_mean] .+ pred_data[:sg_putt_mean_1]) .- pred_data[:SG_putt]
    
    pred_data[:pred_sq_error] = pred_data[:SG_drive_pred].*pred_data[:SG_drive_pred] .+ pred_data[:SG_appr_pred].*pred_data[:SG_appr_pred] .+ pred_data[:SG_short_pred].*pred_data[:SG_short_pred] .+ pred_data[:SG_putt_pred].*pred_data[:SG_putt_pred]
    rmse = sqrt(mean(pred_data[:pred_sq_error]))/2
    drive_rmse = sqrt(mean(pred_data[:SG_drive_pred].*pred_data[:SG_drive_pred]))/2
    appr_rmse = sqrt(mean(pred_data[:SG_appr_pred].*pred_data[:SG_appr_pred]))/2
    short_rmse = sqrt(mean(pred_data[:SG_short_pred].*pred_data[:SG_short_pred]))/2
    putt_rmse = sqrt(mean(pred_data[:SG_putt_pred].*pred_data[:SG_putt_pred]))/2
    
    return rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse,  pred_data[:pred_sq_error]
end


function shuffle_data(df)
    rmd = 0
    train = DataFrame()
    test = DataFrame()
    for i = unique(df[!,:c_idx])
        unq_plyrs = unique(filter(row -> (row.c_idx == i) , df)[!,:p_y_idx])
        n = length(unq_plyrs)
        if n <2
            rmd = rmd + 1
            continue
        else
            idx = shuffle(1:n)
            train_idx = view(idx, 1:floor(Int, train_prcnt*n))
            test_idx = view(idx, (floor(Int, train_prcnt*n)+1):n)
            train_plyrs = unq_plyrs[train_idx,:]
            
            test_plyrs = unq_plyrs[test_idx,:]
            for j = train_plyrs
                append!(train, filter(row -> (row.p_y_idx == j) & (row.c_idx ==i) , df))
            end
            for k = test_plyrs
                append!(test, filter(row -> (row.p_y_idx == k) & (row.c_idx ==i) , df))
            end
        end
    end
        
    return train, test
end

shuffle_data (generic function with 1 method)

# Fit on both

In [12]:
#### PARAMS

dsr_upper = 3
dsr_lower = 0.0
sg_upper = 4
sg_lower = -4

nodsr_pred_errors = []
nodsr_rmse = 0
nodsr_drive_rmse = 0
nodsr_appr_rmse= 0
nodsr_short_rmse = 0
nodsr_putt_rmse = 0

nodsr_p_ests = DataFrame()
nodsr_c_ests = DataFrame()

nodsr_train_error = 0

nodsr_pred_errors_test = []
nodsr_rmse_test = 0
nodsr_drive_rmse_test = 0
nodsr_appr_rmse_test = 0
nodsr_short_rmse_test = 0
nodsr_putt_rmse_test = 0

nodsr_p_ests_test = DataFrame()
nodsr_c_ests_test = DataFrame()

nodsr_train_error_test = 0


real_data_ = readtable("C:/Users/Luke/OneDrive/Research/sg_data/sg_hole_train.csv") # scaled data
real_data_[!, :SG_total] = real_data_[!, :SG_appr] + real_data_[!, :SG_short]  +real_data_[!, :SG_drive] + real_data_[!, :SG_putt] 


pred_errors = []
dsr_rmse = 0
dsr_drive_rmse = 0
dsr_appr_rmse= 0
dsr_short_rmse = 0
dsr_putt_rmse = 0

dsr_p_ests = DataFrame()
dsr_c_ests = DataFrame()
dsr_d_ests = DataFrame()

dsr_train_error = 0

pred_errors_test = []
dsr_rmse_test = 0
dsr_drive_rmse_test = 0
dsr_appr_rmse_test= 0
dsr_short_rmse_test = 0
dsr_putt_rmse_test = 0

dsr_p_ests_test = DataFrame()
dsr_c_ests_test = DataFrame()
dsr_d_ests_test = DataFrame()

dsr_train_error_test = 0

test_rmse_vs_train_df = DataFrame()


# w dsr

for i in 1:num_iterations
    
    println("Iteration: ", i)
    
    # split data
    nodsr_obj_val=0
    n = nrow(real_data_)
    #train_data, test_data = shuffle_data(real_data_)
    train_data = real_data_
    
    # get train data ready
    
    train_plyrs = unique(train_data[!,:plyr_year])
    train_courses = unique(train_data[!,:course_name])
    # need to remove any values in training but not test
    
    n_real = length(train_data[!,:plyr_year])
    num_plyrs = length(unique(train_data[!,:plyr_year]))
    num_courses = length(unique(train_data[!,:course_name]))
    
    idx_X_p_y_idx = DataFrame(X_p_y_idx = 1:num_plyrs, p_y_idx = unique(train_data[!,:p_y_idx]))
    idx_X_c_idx = DataFrame(X_c_idx = 1:num_courses, c_idx = unique(train_data[!,:c_idx]))
    
    train_data = join(train_data, idx_X_p_y_idx, on = :p_y_idx)
    real_data = join(train_data, idx_X_c_idx, on = :c_idx)
    pweight_df_nodsr, cweight_df_nodsr = get_weights_nodsr(real_data)
    

    mod_real = Model(with_optimizer(Ipopt.Optimizer))
   
    @variable(mod_real, p_sg_drive[1:num_plyrs])
    @variable(mod_real, p_sg_appr[1:num_plyrs])
    @variable(mod_real, p_sg_short[1:num_plyrs])
    @variable(mod_real, p_sg_putt[1:num_plyrs])

    @variable(mod_real, c_sg_drive[1:num_courses])
    @variable(mod_real, c_sg_appr[1:num_courses])
    @variable(mod_real, c_sg_short[1:num_courses])
    @variable(mod_real, c_sg_putt[1:num_courses])
    
    p_sg_drive_ = [p_sg_drive[i] for i in real_data[!,:X_p_y_idx]]
    p_sg_appr_ = [p_sg_appr[i] for i in real_data[!,:X_p_y_idx]]
    p_sg_short_ = [p_sg_short[i] for i in real_data[!,:X_p_y_idx]]
    p_sg_putt_ = [p_sg_putt[i] for i in real_data[!,:X_p_y_idx]]

    c_sg_drive_ = [c_sg_drive[i] for i in real_data[!,:X_c_idx]]
    c_sg_appr_ = [c_sg_appr[i] for i in real_data[!,:X_c_idx]]
    c_sg_short_ = [c_sg_short[i] for i in real_data[!,:X_c_idx]]
    c_sg_putt_ = [c_sg_putt[i] for i in real_data[!,:X_c_idx]]
    
    hsg_wmean_drive = @expression(mod_real, sum(c_sg_drive[i]*(filter(row -> row.X_c_idx == i, cweight_df_nodsr)[!,:weight][1]) for i in 1:num_courses))
    hsg_wmean_appr = @expression(mod_real, sum(c_sg_appr[i]*(filter(row -> row.X_c_idx == i, cweight_df_nodsr)[!,:weight][1]) for i in 1:num_courses))
    hsg_wmean_short = @expression(mod_real, sum(c_sg_short[i]*(filter(row -> row.X_c_idx == i, cweight_df_nodsr)[!,:weight][1]) for i in 1:num_courses))
    hsg_wmean_putt = @expression(mod_real, sum(c_sg_putt[i]*(filter(row -> row.X_c_idx == i, cweight_df_nodsr)[!,:weight][1]) for i in 1:num_courses))

    hsg_drive_avg_constr = @constraint(mod_real,  hsg_wmean_drive == 0.0) 
    hsg_appr_avg_constr = @constraint(mod_real,  hsg_wmean_appr == 0.0) 
    hsg_short_avg_constr = @constraint(mod_real,  hsg_wmean_short == 0.0) 
    hsg_putt_avg_constr = @constraint(mod_real,  hsg_wmean_putt == 0.0)

    psg_wmean_drive = @expression(mod_real, sum(p_sg_drive[i]*(filter(row -> row.X_p_y_idx == i, pweight_df_nodsr)[!,:weight][1]) for i in 1:num_plyrs))
    psg_wmean_appr = @expression(mod_real, sum(p_sg_appr[i]*(filter(row -> row.X_p_y_idx == i, pweight_df_nodsr)[!,:weight][1]) for i in 1:num_plyrs))
    psg_wmean_short = @expression(mod_real, sum(p_sg_short[i]*(filter(row -> row.X_p_y_idx == i, pweight_df_nodsr)[!,:weight][1]) for i in 1:num_plyrs))
    psg_wmean_putt = @expression(mod_real, sum(p_sg_putt[i]*(filter(row -> row.X_p_y_idx == i, pweight_df_nodsr)[!,:weight][1]) for i in 1:num_plyrs))

    psg_drive_avg_constr = @constraint(mod_real,  psg_wmean_drive == 0.0) 
    psg_appr_avg_constr = @constraint(mod_real,  psg_wmean_appr == 0.0) 
    psg_short_avg_constr = @constraint(mod_real,  psg_wmean_short == 0.0) 
    psg_putt_avg_constr = @constraint(mod_real,  psg_wmean_putt == 0.0)

    sg_drive_constr = @constraint(mod_real, [i=1:num_courses], -5 <= c_sg_drive[i] <= 5) 
    sg_appr_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_appr[i] <= 5) 
    sg_short_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_short[i] <= 5) 
    sg_putt_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_putt[i] <= 5) 

    psg_drive_constr = @constraint(mod_real, [i=1:num_plyrs], -5 <= p_sg_drive[i] <= 5) 
    psg_appr_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_appr[i] <= 5) 
    psg_short_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_short[i] <= 5) 
    psg_putt_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_putt[i] <= 5) 
    
    @NLobjective(mod_real, Min, sum( (p_sg_drive_[i]*p_sg_drive_[i] + 2*p_sg_drive_[i]*c_sg_drive_[i] + c_sg_drive_[i]*c_sg_drive_[i] - 2*p_sg_drive_[i]*real_data[!,:SG_drive][i] - 2*c_sg_drive_[i]*real_data[!,:SG_drive][i] + real_data[!,:SG_drive][i]*real_data[!,:SG_drive][i]) + (p_sg_appr_[i]*p_sg_appr_[i] + 2*p_sg_appr_[i]*c_sg_appr_[i] + c_sg_appr_[i]*c_sg_appr_[i] - 2*p_sg_appr_[i]*real_data[!,:SG_appr][i] - 2*c_sg_appr_[i]*real_data[!,:SG_appr][i] + real_data[!,:SG_appr][i]*real_data[!,:SG_appr][i]) + (p_sg_short_[i]*p_sg_short_[i] + 2*p_sg_short_[i]*c_sg_short_[i] + c_sg_short_[i]*c_sg_short_[i] - 2*p_sg_short_[i]*real_data[!,:SG_short][i] - 2*c_sg_short_[i]*real_data[!,:SG_short][i] + real_data[!,:SG_short][i]*real_data[!,:SG_short][i]) + (p_sg_putt_[i]*p_sg_putt_[i] + 2*p_sg_putt_[i]*c_sg_putt_[i] + c_sg_putt_[i]*c_sg_putt_[i] - 2*p_sg_putt_[i]*real_data[!,:SG_putt][i] - 2*c_sg_putt_[i]*real_data[!,:SG_putt][i] + real_data[!,:SG_putt][i]*real_data[!,:SG_putt][i]) for i in 1:n_real)/( n_real))
    status_real = optimize!(mod_real)
    
    c_idx_df = DataFrame(idx = unique(real_data[!,:c_idx]), X_idx = unique(real_data[!,:X_c_idx]), course_name = unique(real_data[!,:course_name]))
    p_y_idx_df = DataFrame(idx = unique(real_data[!,:p_y_idx]), X_idx = unique(real_data[!,:X_p_y_idx]), plyr_year = unique(real_data[!,:plyr_year]))

    p_d_label = [value.(p_sg_drive)[i] for i in p_y_idx_df[!,:X_idx]]
    p_a_label = [value.(p_sg_appr)[i] for i in p_y_idx_df[!,:X_idx]]
    p_s_label = [value.(p_sg_short)[i] for i in p_y_idx_df[!,:X_idx]]
    p_p_label = [value.(p_sg_putt)[i] for i in p_y_idx_df[!,:X_idx]]

    c_d_label = [value.(c_sg_drive)[i] for i in c_idx_df[!,:X_idx]]
    c_a_label = [value.(c_sg_appr)[i] for i in c_idx_df[!,:X_idx]]
    c_s_label = [value.(c_sg_short)[i] for i in c_idx_df[!,:X_idx]]
    c_p_label = [value.(c_sg_putt)[i] for i in c_idx_df[!,:X_idx]]


    p_rounds = [(filter(row -> row.X_p_y_idx == i, pweight_df_nodsr)[!,:weight][1])*n_real for i in p_y_idx_df[!,:X_idx]]
    c_rounds = [(filter(row -> row.X_c_idx == i, cweight_df_nodsr)[!,:weight][1])*n_real for i in c_idx_df[!,:X_idx]]
    nodsr_obj_val = nodsr_obj_val + sqrt(objective_value(mod_real))/2
    
    nodsr_c_sg_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], sg_drive = c_d_label, sg_appr = c_a_label, sg_short = c_s_label, sg_putt = c_p_label,num_rounds = c_rounds)

    nodsr_p_final_df = DataFrame(p_y_idx = p_y_idx_df[!,:idx], plyr_year = p_y_idx_df[!,:plyr_year], sg_drive = p_d_label, sg_appr = p_a_label, sg_short = p_s_label, sg_putt = p_p_label, num_rounds = p_rounds)

    
    append!(nodsr_c_ests, nodsr_c_sg_final_df)
    append!(nodsr_p_ests, nodsr_p_final_df)
    
    new_rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse, preds = test_rmse_nodsr(nodsr_p_final_df, nodsr_c_sg_final_df,  train_data)
    nodsr_rmse = nodsr_rmse + new_rmse
    nodsr_drive_rmse = nodsr_drive_rmse + drive_rmse
    nodsr_appr_rmse = nodsr_appr_rmse + appr_rmse
    nodsr_short_rmse = nodsr_short_rmse + short_rmse
    nodsr_putt_rmse = nodsr_putt_rmse + putt_rmse
    
    
    println("#### OUTPUT STARTS HERE ######")
    println( "\nIteration RMSE: ", new_rmse)
    
    append!(nodsr_pred_errors, preds)
    nodsr_train_error = nodsr_train_error + nodsr_obj_val
    println("Iteration ",i)
    println()
    println("Iteration RMSE: ", new_rmse)
    println("\nTraining error: ", nodsr_train_error/i)
    println("\nAverage RMSES: ")
    println("\nTotal RMSE: ", nodsr_rmse/i, "\n")
    println("\nDrive RMSE: ", nodsr_drive_rmse/i, "\n")
    println("\nAppr RMSE: ", nodsr_appr_rmse/i, "\n")
    println("\nShort RMSE: ", nodsr_short_rmse/i, "\n")
    println("\nPutt RMSE: ", nodsr_putt_rmse/i, "\n")
    
    ####
    obj_val=0
    
    pweight_df, cweight_df = get_weights(real_data)
    
    
    # driver
    #println("here1")
    mod_real = Model(with_optimizer(Ipopt.Optimizer))

    @variable(mod_real, p_sg_drive[1:num_plyrs])
    @variable(mod_real, c_sg_drive[1:num_courses])
    @variable(mod_real, c_dsr_drive[1:num_courses])

    p_sg_drive_ = [p_sg_drive[i] for i in real_data[!,:X_p_y_idx]]
    c_sg_drive_ = [c_sg_drive[i] for i in real_data[!,:X_c_idx]]
    c_dsr_drive_ = [c_dsr_drive[i] for i in real_data[!,:X_c_idx]]
  
    dsr_wmean_drive = @expression(mod_real, sum(c_dsr_drive[i]*(filter(row -> row.X_c_idx == i, cweight_df)[!,:weight][1]) for i in 1:num_courses)) # weighted
    dsr_drive_avg_constr = @constraint(mod_real,  dsr_wmean_drive == 1.0) 

    hsg_wmean_drive = @expression(mod_real, sum(c_sg_drive[i]*(filter(row -> row.X_c_idx == i, cweight_df)[!,:weight][1]) for i in 1:num_courses)) # weighted
    hsg_drive_avg_constr = @constraint(mod_real,  hsg_wmean_drive == 0.0) 


    psg_wmean_drive = @expression(mod_real, sum(p_sg_drive[i]*(filter(row -> row.X_p_y_idx == i, pweight_df)[!,:weight][1]) for i in 1:num_plyrs)) # weighted
    psg_drive_avg_constr = @constraint(mod_real,  psg_wmean_drive == 0.0) 

    dsr_drive_constr = @constraint(mod_real, [i=1:num_courses], dsr_lower <= c_dsr_drive[i] <= dsr_upper) 

    sg_dpx_constr = @constraint(mod_real, [i=1:num_courses], sg_lower <= c_sg_drive[i] <= sg_upper) 
    psg_dpx_constr = @constraint(mod_real, [i=1:num_plyrs], sg_lower <= p_sg_drive[i] <= sg_upper)

    # p*d - c
    @NLobjective(mod_real, Min, sum( (p_sg_drive_[i]*p_sg_drive_[i]*c_dsr_drive_[i]*c_dsr_drive_[i] + 2*p_sg_drive_[i]*c_dsr_drive_[i]*c_sg_drive_[i] + c_sg_drive_[i]*c_sg_drive_[i]- 2*p_sg_drive_[i]*c_dsr_drive_[i]*real_data[!,:SG_drive][i] - 2*c_sg_drive_[i]*real_data[!,:SG_drive][i] + real_data[!,:SG_drive][i]*real_data[!,:SG_drive][i]) for i in 1:n_real)/(n_real))

    status_real = optimize!(mod_real)

    c_idx_df = DataFrame(idx = unique(real_data[!,:c_idx]),X_idx = unique(real_data[!,:X_c_idx]), course_name = unique(real_data[!,:course_name]))
    p_y_idx_df = DataFrame(idx = unique(real_data[!,:p_y_idx]), X_idx = unique(real_data[!,:X_p_y_idx]), plyr_year = unique(real_data[!,:plyr_year]))

    p_d_label = [value.(p_sg_drive)[i] for i in p_y_idx_df[!,:X_idx]]

    c_d_label = [value.(c_sg_drive)[i] for i in c_idx_df[!,:X_idx]]
    

    cd_d_label = [value.(c_dsr_drive)[i] for i in c_idx_df[!,:X_idx]]

    rounds = [(filter(row -> row.X_p_y_idx == i, pweight_df)[!,:weight][1])*n_real for i in p_y_idx_df[!,:X_idx]]
    obj_val = obj_val + objective_value(mod_real)

    c_sg_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], sg_drive = c_d_label)
    c_dsr_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], dsr_drive = cd_d_label)

    p_final_df = DataFrame(p_y_idx = p_y_idx_df[!,:idx], plyr_year = p_y_idx_df[!,:plyr_year], sg_drive = p_d_label, num_rounds = rounds)

    # appr

    mod_real = Model(with_optimizer(Ipopt.Optimizer))

    @variable(mod_real, p_sg_appr[1:num_plyrs])
    @variable(mod_real, c_sg_appr[1:num_courses])
    @variable(mod_real, c_dsr_appr[1:num_courses])

    p_sg_appr_ = [p_sg_appr[i] for i in real_data[!,:X_p_y_idx]]
    c_sg_appr_ = [c_sg_appr[i] for i in real_data[!,:X_c_idx]]
    c_dsr_appr_ = [c_dsr_appr[i] for i in real_data[!,:X_c_idx]]

    
    dsr_wmean_appr = @expression(mod_real, sum(c_dsr_appr[i]*(filter(row -> row.X_c_idx == i, cweight_df)[!,:weight][1]) for i in 1:num_courses)) # weighted
    dsr_appr_avg_constr = @constraint(mod_real,  dsr_wmean_appr == 1.0) 

    hsg_wmean_appr = @expression(mod_real, sum(c_sg_appr[i]*(filter(row -> row.X_c_idx == i, cweight_df)[!,:weight][1]) for i in 1:num_courses)) # weighted
    hsg_appr_avg_constr = @constraint(mod_real,  hsg_wmean_appr == 0.0) 

    psg_wmean_appr = @expression(mod_real, sum(p_sg_appr[i]*(filter(row -> row.X_p_y_idx == i, pweight_df)[!,:weight][1]) for i in 1:num_plyrs)) #weighted
    psg_appr_avg_constr = @constraint(mod_real,  psg_wmean_appr == 0.0)

    dsr_appr_constr = @constraint(mod_real, [i=1:num_courses], dsr_lower <= c_dsr_appr[i] <= dsr_upper) 
    
    # general
    sg_apx_constr = @constraint(mod_real, [i=1:num_courses],  sg_lower <= c_sg_appr[i] <= sg_upper) 
    psg_apx_constr = @constraint(mod_real, [i=1:num_plyrs], sg_lower <= p_sg_appr[i] <= sg_upper)

    # p*d - c
    @NLobjective(mod_real, Min, sum( (p_sg_appr_[i]*p_sg_appr_[i]*c_dsr_appr_[i]*c_dsr_appr_[i] + 2*p_sg_appr_[i]*c_dsr_appr_[i]*c_sg_appr_[i] + c_sg_appr_[i]*c_sg_appr_[i]- 2*p_sg_appr_[i]*c_dsr_appr_[i]*real_data[!,:SG_appr][i] - 2*c_sg_appr_[i]*real_data[!,:SG_appr][i] + real_data[!,:SG_appr][i]*real_data[!,:SG_appr][i]) for i in 1:n_real)/(n_real))
    
    status_real = optimize!(mod_real)

    p_a_label = [value.(p_sg_appr)[i] for i in p_y_idx_df[!,:X_idx]]

    c_a_label = [value.(c_sg_appr)[i] for i in c_idx_df[!,:X_idx]]

    cd_a_label = [value.(c_dsr_appr)[i] for i in c_idx_df[!,:X_idx]]

    obj_val = obj_val + objective_value(mod_real)

    c_sg_final_df[:sg_appr] = c_a_label
    c_dsr_final_df[:dsr_appr] = cd_a_label

    p_final_df[:sg_appr] = p_a_label
    
    mod_real = Model(with_optimizer(Ipopt.Optimizer))

    @variable(mod_real, p_sg_short[1:num_plyrs])
    @variable(mod_real, c_sg_short[1:num_courses])
    @variable(mod_real, c_dsr_short[1:num_courses])

    p_sg_short_ = [p_sg_short[i] for i in real_data[!,:X_p_y_idx]]
    c_sg_short_ = [c_sg_short[i] for i in real_data[!,:X_c_idx]]
    c_dsr_short_ = [c_dsr_short[i] for i in real_data[!,:X_c_idx]]

    dsr_wmean_short = @expression(mod_real, sum(c_dsr_short[i]*(filter(row -> row.X_c_idx == i, cweight_df)[!,:weight][1]) for i in 1:num_courses)) # weighted
    dsr_short_avg_constr = @constraint(mod_real,  dsr_wmean_short == 1.0) 

    hsg_wmean_short = @expression(mod_real, sum(c_sg_short[i]*(filter(row -> row.X_c_idx == i, cweight_df)[!,:weight][1]) for i in 1:num_courses)) # weighted
    hsg_short_avg_constr = @constraint(mod_real,  hsg_wmean_short == 0.0) 

    psg_wmean_short = @expression(mod_real, sum(p_sg_short[i]*(filter(row -> row.X_p_y_idx == i, pweight_df)[!,:weight][1]) for i in 1:num_plyrs)) # weighted
    psg_short_avg_constr = @constraint(mod_real,  psg_wmean_short == 0.0) 

    dsr_short_constr = @constraint(mod_real, [i=1:num_courses], dsr_lower <= c_dsr_short[i] <= dsr_upper) 

    # general
    sg_spx_constr = @constraint(mod_real, [i=1:num_courses], sg_lower <= c_sg_short[i] <= sg_upper) 
    psg_spx_constr = @constraint(mod_real, [i=1:num_plyrs], sg_lower <= p_sg_short[i] <= sg_upper)

    # p*d - c
    @NLobjective(mod_real, Min, sum( (p_sg_short_[i]*p_sg_short_[i]*c_dsr_short_[i]*c_dsr_short_[i] + 2*p_sg_short_[i]*c_dsr_short_[i]*c_sg_short_[i] + c_sg_short_[i]*c_sg_short_[i]- 2*p_sg_short_[i]*c_dsr_short_[i]*real_data[!,:SG_short][i] - 2*c_sg_short_[i]*real_data[!,:SG_short][i] + real_data[!,:SG_short][i]*real_data[!,:SG_short][i]) for i in 1:n_real)/(n_real))
    
    status_real = optimize!(mod_real)

    p_s_label = [value.(p_sg_short)[i] for i in p_y_idx_df[!,:X_idx]]

    c_s_label = [value.(c_sg_short)[i] for i in c_idx_df[!,:X_idx]]

    cd_s_label = [value.(c_dsr_short)[i] for i in c_idx_df[!,:X_idx]]
    
    obj_val = obj_val + objective_value(mod_real)

    c_sg_final_df[:sg_short] = c_s_label
    c_dsr_final_df[:dsr_short] = cd_s_label

    p_final_df[:sg_short] = p_s_label
    
    # putt
    mod_real = Model(with_optimizer(Ipopt.Optimizer))

    @variable(mod_real, p_sg_putt[1:num_plyrs])
    @variable(mod_real, c_sg_putt[1:num_courses])
    @variable(mod_real, c_dsr_putt[1:num_courses])

    p_sg_putt_ = [p_sg_putt[i] for i in real_data[!,:X_p_y_idx]]
    c_sg_putt_ = [c_sg_putt[i] for i in real_data[!,:X_c_idx]]
    c_dsr_putt_ = [c_dsr_putt[i] for i in real_data[!,:X_c_idx]]

    dsr_wmean_putt = @expression(mod_real, sum(c_dsr_putt[i]*(filter(row -> row.X_c_idx == i, cweight_df)[!,:weight][1]) for i in 1:num_courses)) # weighted
    dsr_putt_avg_constr = @constraint(mod_real,  dsr_wmean_putt == 1.0) 

    hsg_wmean_putt = @expression(mod_real, sum(c_sg_putt[i]*(filter(row -> row.X_c_idx == i, cweight_df)[!,:weight][1]) for i in 1:num_courses)) # weighted
    hsg_putt_avg_constr = @constraint(mod_real,  hsg_wmean_putt == 0.0) 

    psg_wmean_putt = @expression(mod_real, sum(p_sg_putt[i]*(filter(row -> row.X_p_y_idx == i, pweight_df)[!,:weight][1]) for i in 1:num_plyrs)) # weighted
    psg_putt_avg_constr = @constraint(mod_real,  psg_wmean_putt == 0.0) 

    dsr_putt_constr = @constraint(mod_real, [i=1:num_courses], dsr_lower <= c_dsr_putt[i] <= dsr_upper) 
    
    
    # general 
    sg_ppx_constr = @constraint(mod_real, [i=1:num_courses], sg_lower <= c_sg_putt[i] <= sg_upper) 
    psg_ppx_constr = @constraint(mod_real, [i=1:num_plyrs], sg_lower <= p_sg_putt[i] <= sg_upper)


    # p*d - c
    @NLobjective(mod_real, Min, sum( (p_sg_putt_[i]*p_sg_putt_[i]*c_dsr_putt_[i]*c_dsr_putt_[i] + 2*p_sg_putt_[i]*c_dsr_putt_[i]*c_sg_putt_[i] + c_sg_putt_[i]*c_sg_putt_[i]- 2*p_sg_putt_[i]*c_dsr_putt_[i]*real_data[!,:SG_putt][i] - 2*c_sg_putt_[i]*real_data[!,:SG_putt][i] + real_data[!,:SG_putt][i]*real_data[!,:SG_putt][i]) for i in 1:n_real)/(n_real))
    

    status_real = optimize!(mod_real)


    p_p_label = [value.(p_sg_putt)[i] for i in p_y_idx_df[!,:X_idx]]

    c_p_label = [value.(c_sg_putt)[i] for i in c_idx_df[!,:X_idx]]

    cd_p_label = [value.(c_dsr_putt)[i] for i in c_idx_df[!,:X_idx]]

    obj_val = obj_val + objective_value(mod_real)
    
    ## MAKE THIS MORE EFFICIENT BUT THIS SHOULD WORK
    obj_val = sqrt(obj_val)/2

    c_sg_final_df[:sg_putt] = c_p_label
    c_dsr_final_df[:dsr_putt] = cd_p_label

    p_final_df[:sg_putt] = p_p_label
    
    
    
    append!(dsr_c_ests, c_sg_final_df)
    append!(dsr_p_ests, p_final_df)
    append!(dsr_d_ests, c_dsr_final_df)
    
    
    new_rmse, drive_rmse, appr_rmse, short_rmse, putt_rmse, preds = test_rmse(p_final_df, c_sg_final_df, c_dsr_final_df, train_data)
    dsr_rmse = dsr_rmse + new_rmse
    dsr_drive_rmse = dsr_drive_rmse + drive_rmse
    dsr_appr_rmse = dsr_appr_rmse + appr_rmse
    dsr_short_rmse = dsr_short_rmse + short_rmse
    dsr_putt_rmse = dsr_putt_rmse + putt_rmse
    
    dsr_train_error = dsr_train_error + obj_val
    
    append!(pred_errors, preds)
    
    println("Iteration ",i,"\nTraining error: ", obj_val)
    println()
    println("\nIteration RMSE: ", new_rmse)
    println("\n Training error: ", dsr_train_error/i)
    println("Average RMSES: ")
    println("\nTotal RMSE: ", dsr_rmse/i)
    println("\nDrive RMSE: ", dsr_drive_rmse/i)
    println("\nAppr RMSE: ", dsr_appr_rmse/i )
    println("\nShort RMSE: ", dsr_short_rmse/i)
    println("\nPutt RMSE: ", dsr_putt_rmse/i)

            end 

Iteration: 

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[12]:31
└ @ Core In[12]:31


1
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     9976
Number of nonzeros in inequality constraint Jacobian.:    19952
Number of nonzeros in Lagrangian Hessian.............:   106216

Total number of variables............................:     9976
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:    19952
        inequality constraints with only lower bounds:     9976
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     9976

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = test_rmse_nodsr(::DataFrame, ::DataFrame, ::DataFrame) at In[11]:57
└ @ Main .\In[11]:57
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = test_rmse_nodsr(::DataFrame, ::DataFrame, ::DataFrame) at In[11]:57
└ @ Main .\In[11]:57
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = test_rmse_nodsr(::DataFrame, ::DataFrame, ::DataFrame) at In[11]:57
└ @ Main .\In[11]:57
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = test_rmse_nodsr(::DataFrame, ::DataFrame, ::DataFrame) at In[11]:57
└ @ Main .\In[11]:57
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = test_rms

#### OUTPUT STARTS HERE ######

Iteration RMSE: 1.3803669624536277
Iteration 1

Iteration RMSE: 1.3803669624536277

Training error: 1.3803669624536377

Average RMSES: 

Total RMSE: 1.3803669624536277


Drive RMSE: 0.5409405675798002


Appr RMSE: 0.7875880793113959


Short RMSE: 0.5395529746306487


Putt RMSE: 0.8374866316985026



┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = test_rmse_nodsr(::DataFrame, ::DataFrame, ::DataFrame) at In[11]:62
└ @ Main .\In[11]:62
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = test_rmse_nodsr(::DataFrame, ::DataFrame, ::DataFrame) at In[11]:62
└ @ Main .\In[11]:62
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = test_rmse_nodsr(::DataFrame, ::DataFrame, ::DataFrame) at In[11]:62
└ @ Main .\In[11]:62
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = test_rmse_nodsr(::DataFrame, ::DataFrame, ::DataFrame) at In[11]:63
└ @ Main .\In[11]:63
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = test_rms

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2554
Number of nonzeros in inequality constraint Jacobian.:     5108
Number of nonzeros in Lagrangian Hessian.............:    50734

Total number of variables............................:     2554
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequality constraints...............:     5108
        inequality constraints with only lower bounds:     2554
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     2554

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[12]:298
└ @ Core In[12]:298
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[12]:299
└ @ Core In[12]:299
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[12]:301
└ @ Core In[12]:301


OutOfMemoryError: OutOfMemoryError()

In [8]:
CSV.write("model_data/dsr_p_ests_0814_train.csv", dsr_p_ests)
CSV.write("model_data/dsr_c_ests_0814_train.csv", dsr_c_ests)
CSV.write("model_data/dsr_d_ests_0814_train.csv", dsr_d_ests)

CSV.write("model_data/nodsr_p_ests_0814_train.csv", nodsr_p_ests)
CSV.write("model_data/nodsr_c_ests_0814_train.csv", nodsr_c_ests)

"model_data/nodsr_c_ests_0814_train.csv"

# Analysis 

In [9]:
### DSR 0.1 to 3
### SG -4 to 4

min_n = min(length(nodsr_pred_errors),length(pred_errors))
dsr_ = pred_errors[1:min_n]
nodsr_ = nodsr_pred_errors[1:min_n]

print("Average train error for nodsr model: ", nodsr_train_error/num_iterations, "\n")
print("Average train error for dsr model: ", dsr_train_error/num_iterations, "\n\n")

print("Average prediction test error for nodsr model: ", nodsr_rmse/num_iterations, "\n")

print("Average prediction test error for dsr model: ", dsr_rmse/num_iterations, "\n")


print("Average prediction test DRIVE error for nodsr model: ", nodsr_drive_rmse/num_iterations, "\n")
print("Average prediction test DRIVE error for dsr model: ", dsr_drive_rmse/num_iterations, "\n")

print("Average prediction test APPR error for nodsr model: ", nodsr_appr_rmse/num_iterations, "\n")
print("Average prediction test APPR error for dsr model: ", dsr_appr_rmse/num_iterations, "\n")

print("Average prediction test SHORT error for nodsr model: ", nodsr_short_rmse/num_iterations, "\n")
print("Average prediction test SHORT error for dsr model: ", dsr_short_rmse/num_iterations, "\n")

print("Average prediction test PUTT error for nodsr model: ", nodsr_putt_rmse/num_iterations, "\n")
print("Average prediction test PUTT error for dsr model: ", dsr_putt_rmse/num_iterations, "\n")

Average train error for nodsr model: 1.3803669624536374
Average train error for dsr model: 1.379150214585728

Average prediction test error for nodsr model: 1.3803669624536279
Average prediction test error for dsr model: 1.37915021458573
Average prediction test DRIVE error for nodsr model: 0.5409405675798001
Average prediction test DRIVE error for dsr model: 0.5406498792934905
Average prediction test APPR error for nodsr model: 0.787588079311396
Average prediction test APPR error for dsr model: 0.786772299488696
Average prediction test SHORT error for nodsr model: 0.5395529746306487
Average prediction test SHORT error for dsr model: 0.5390029325759419
Average prediction test PUTT error for nodsr model: 0.8374866316985026
Average prediction test PUTT error for dsr model: 0.8367904216968569


# Explore